In [3]:
path = "E:\\data_bone\\11-a+b_swift_cut_正確_V2_踢盲檢\\front"
path2 = "E:\\data_bone\\11-只有盲檢\\front"
concat_type = "concat1_踢盲檢_測盲檢"

In [4]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import os
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
import math
from tensorflow.keras.models import Model

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def create_front_extract():
    pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def create_side_extract():
    pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def multi_input_generator(front_gen, side_gen):
    while True:
        front_batch, y1 = next(front_gen)
        side_batch, y2 = next(side_gen)
        assert (y1 == y2).all(), "Label mismatch!"  # 確保標籤一致
        yield ([front_batch, side_batch], y1)


# def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):
def train_cross_validation(image_dir, image_dir2, n_splits=5, class_count=2):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================

    ## load data and  labels
    # =========================
    data2 = load_path(image_dir2, class_count)
    labels = []
    filepaths = []
    for row in data2:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images2 = pd.concat([filepaths, labels], axis=1)
    # =========================

    ## split image
    # =========================
    # train_df_front, val_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
    # train_df_front, test_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
    train_df_front = images.sample(frac=1, random_state=42)
    test_df_front = images2
    print("Training set label distribution:\n", train_df_front['Label'].value_counts(normalize=False))
    print("Test set label distribution:\n", test_df_front['Label'].value_counts(normalize=False))
    # =========================

    preprocessing_function_chosen_front = tf.keras.applications.resnet50.preprocess_input
    preprocessing_function_chosen_side = tf.keras.applications.efficientnet.preprocess_input


    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []

    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df_front, train_df_front['Label']):
        k_fold_train_front = train_df_front.iloc[train_index]
        k_fold_val_front = train_df_front.iloc[val_index]

        # front images
        # =========================
        train_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                                preprocessing_function=preprocessing_function_chosen_front)
        test_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_front)

        train_images_front = train_generator_front.flow_from_dataframe(
            dataframe=k_fold_train_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        val_images_front = train_generator_front.flow_from_dataframe(
            dataframe=k_fold_val_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        test_images_front = test_generator_front.flow_from_dataframe(
            dataframe=test_df_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=32,
            shuffle=False
        )
        # =========================   

        # side images
        # =========================
        train_df_side = train_df_front.copy()
        test_df_side = test_df_front.copy()
        train_df_side.loc[:, "Filepath"] = train_df_front["Filepath"].str.replace("front", "side")
        test_df_side.loc[:, "Filepath"] = test_df_side["Filepath"].str.replace("front", "side")

        k_fold_train_side = train_df_side.iloc[train_index]
        k_fold_val_side = train_df_side.iloc[val_index]

        train_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                                preprocessing_function=preprocessing_function_chosen_side)
        test_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_side)

        train_images_side = train_generator_side.flow_from_dataframe(
            dataframe=k_fold_train_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        val_images_side = train_generator_side.flow_from_dataframe(
            dataframe=k_fold_val_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        test_images_side = test_generator_side.flow_from_dataframe(
            dataframe=test_df_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=32,
            shuffle=False
        )
        # ========================= 
        
        # load model
        # =========================
        # input 
        input_front = Input(shape=(224, 224, 3), name="AP(Mortise)")
        input_side = Input(shape=(224, 224, 3), name="Lateral")

        # model
        model_front = create_front_extract()
        model_side = create_side_extract()

        features_front = model_front(input_front)
        features_side = model_side(input_side)

        features_front_x = tf.keras.layers.Dense(128, activation='relu', name='AP_dense_128')(features_front)
        features_front_x = tf.keras.layers.Dense(50, activation='relu', name='AP_dense_50')(features_front_x)

        features_side_x = tf.keras.layers.Dense(128, activation='relu', name='Lateral_dense_128')(features_side)
        features_side_x = tf.keras.layers.Dense(50, activation='relu', name='Lateral_dense_50')(features_side_x)

        fused_features = Concatenate(name="feature_fusion")([features_front_x, features_side_x])
        # fused_features = tf.keras.layers.Dense(50, activation='relu', name='fusion_dense_50')(fused_features)
        fused_features = Dropout(0.1)(fused_features)

        final_output = Dense(class_count, activation='sigmoid', name='output_layer')(fused_features)
        multi_view_model = None
        multi_view_model = Model(
            inputs=[input_front, input_side],
            outputs=final_output
        )
        multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        multi_view_model.summary()
        # print(model.summary())
        # =========================

        train_generator = multi_input_generator(train_images_front, train_images_side)
        val_generator = multi_input_generator(val_images_front, val_images_side)
        
        ## compile and evaluate
        # =========================

        print("-------Training_" + concat_type + "-------")
        multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
        batch_size = 64
        ## early stop 
        # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
        # multi_view_model.fit(train_generator, validation_data=val_generator, callbacks=[early_stopping], epochs=30,
        #                             steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
        #                             validation_steps= math.ceil(val_images_front.samples / batch_size))
        ## no early stop
        multi_view_model.fit(train_generator, validation_data=val_generator, epochs=30,
                            steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
                            validation_steps= math.ceil(val_images_front.samples / batch_size))

        # =========================

        test_generator = multi_input_generator(test_images_front, test_images_side)
        batch_size=32
        pred = multi_view_model.predict(test_generator,  steps=math.ceil(test_images_front.samples / batch_size))
        predicted_labels = np.argmax(pred, axis=1)

        # =========================
            
        ## print results
        # =========================
        acc = accuracy_score(test_images_front.labels, predicted_labels)

        test_acc.append(np.round(acc, 2))
        f1 = f1_score(test_images_front.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

# =========================




results = []
results = train_cross_validation(image_dir=path,image_dir2=path2, class_count=3)


print(f"acc mean = {results['acc_mean']}, std = {results['acc_std']}, test_acc = {results['test_acc']}")
print(f"f1  mean = {results['f1_mean']}, std = {results['f1_std']}, test_f1 = {results['test_f1']}")
  

Training set label distribution:
 0    126
2     87
1     84
Name: Label, dtype: int64
Test set label distribution:
 2    35
0    34
1    31
Name: Label, dtype: int64
Found 237 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Found 237 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AP(Mortise) (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Lateral (InputLayer)            [(None, 224, 224, 3) 0        

In [5]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import os
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
import math
from tensorflow.keras.models import Model

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def create_front_extract():
    pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def create_side_extract():
    pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def multi_input_generator(front_gen, side_gen):
    while True:
        front_batch, y1 = next(front_gen)
        side_batch, y2 = next(side_gen)
        assert (y1 == y2).all(), "Label mismatch!"  # 確保標籤一致
        yield ([front_batch, side_batch], y1)


# def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):
def train_cross_validation(image_dir, image_dir2, n_splits=5, class_count=2):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================

    ## load data and  labels
    # =========================
    data2 = load_path(image_dir2, class_count)
    labels = []
    filepaths = []
    for row in data2:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images2 = pd.concat([filepaths, labels], axis=1)
    # =========================

    ## split image
    # =========================
    # train_df_front, val_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
    # train_df_front, test_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
    train_df_front = images.sample(frac=1, random_state=42)
    test_df_front = images2
    print("Training set label distribution:\n", train_df_front['Label'].value_counts(normalize=False))
    print("Test set label distribution:\n", test_df_front['Label'].value_counts(normalize=False))
    # =========================

    preprocessing_function_chosen_front = tf.keras.applications.resnet50.preprocess_input
    preprocessing_function_chosen_side = tf.keras.applications.efficientnet.preprocess_input


    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []

    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df_front, train_df_front['Label']):
        k_fold_train_front = train_df_front.iloc[train_index]
        k_fold_val_front = train_df_front.iloc[val_index]

        # front images
        # =========================
        train_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                                preprocessing_function=preprocessing_function_chosen_front)
        test_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_front)

        train_images_front = train_generator_front.flow_from_dataframe(
            dataframe=k_fold_train_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        val_images_front = train_generator_front.flow_from_dataframe(
            dataframe=k_fold_val_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        test_images_front = test_generator_front.flow_from_dataframe(
            dataframe=test_df_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=32,
            shuffle=False
        )
        # =========================   

        # side images
        # =========================
        train_df_side = train_df_front.copy()
        test_df_side = test_df_front.copy()
        train_df_side.loc[:, "Filepath"] = train_df_front["Filepath"].str.replace("front", "side")
        test_df_side.loc[:, "Filepath"] = test_df_side["Filepath"].str.replace("front", "side")

        k_fold_train_side = train_df_side.iloc[train_index]
        k_fold_val_side = train_df_side.iloc[val_index]

        train_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                                preprocessing_function=preprocessing_function_chosen_side)
        test_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_side)

        train_images_side = train_generator_side.flow_from_dataframe(
            dataframe=k_fold_train_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        val_images_side = train_generator_side.flow_from_dataframe(
            dataframe=k_fold_val_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        test_images_side = test_generator_side.flow_from_dataframe(
            dataframe=test_df_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=32,
            shuffle=False
        )
        # ========================= 
        
        # load model
        # =========================
        # input 
        input_front = Input(shape=(224, 224, 3), name="AP(Mortise)")
        input_side = Input(shape=(224, 224, 3), name="Lateral")

        # model
        model_front = create_front_extract()
        model_side = create_side_extract()

        features_front = model_front(input_front)
        features_side = model_side(input_side)

        features_front_x = tf.keras.layers.Dense(128, activation='relu', name='AP_dense_128')(features_front)
        features_front_x = tf.keras.layers.Dense(50, activation='relu', name='AP_dense_50')(features_front_x)

        features_side_x = tf.keras.layers.Dense(128, activation='relu', name='Lateral_dense_128')(features_side)
        features_side_x = tf.keras.layers.Dense(50, activation='relu', name='Lateral_dense_50')(features_side_x)

        fused_features = Concatenate(name="feature_fusion")([features_front_x, features_side_x])
        # fused_features = tf.keras.layers.Dense(50, activation='relu', name='fusion_dense_50')(fused_features)
        fused_features = Dropout(0.1)(fused_features)

        final_output = Dense(class_count, activation='sigmoid', name='output_layer')(fused_features)
        multi_view_model = None
        multi_view_model = Model(
            inputs=[input_front, input_side],
            outputs=final_output
        )
        multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        multi_view_model.summary()
        # print(model.summary())
        # =========================

        train_generator = multi_input_generator(train_images_front, train_images_side)
        val_generator = multi_input_generator(val_images_front, val_images_side)
        
        ## compile and evaluate
        # =========================

        print("-------Training_" + concat_type + "-------")
        multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
        batch_size = 64
        ## early stop 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        multi_view_model.fit(train_generator, validation_data=val_generator, callbacks=[early_stopping], epochs=30,
                                    steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
                                    validation_steps= math.ceil(val_images_front.samples / batch_size))
        ## no early stop
        # multi_view_model.fit(train_generator, validation_data=val_generator, epochs=30,
        #                     steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
        #                     validation_steps= math.ceil(val_images_front.samples / batch_size))

        # =========================

        test_generator = multi_input_generator(test_images_front, test_images_side)
        batch_size=32
        pred = multi_view_model.predict(test_generator,  steps=math.ceil(test_images_front.samples / batch_size))
        predicted_labels = np.argmax(pred, axis=1)

        # =========================
            
        ## print results
        # =========================
        acc = accuracy_score(test_images_front.labels, predicted_labels)

        test_acc.append(np.round(acc, 2))
        f1 = f1_score(test_images_front.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

# =========================




results = []
results = train_cross_validation(image_dir=path,image_dir2=path2, class_count=3)


print(f"acc mean = {results['acc_mean']}, std = {results['acc_std']}, test_acc = {results['test_acc']}")
print(f"f1  mean = {results['f1_mean']}, std = {results['f1_std']}, test_f1 = {results['test_f1']}")
  

Training set label distribution:
 0    126
2     87
1     84
Name: Label, dtype: int64
Test set label distribution:
 2    35
0    34
1    31
Name: Label, dtype: int64
Found 237 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Found 237 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AP(Mortise) (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Lateral (InputLayer)            [(None, 224, 224, 3) 0        

In [6]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import os
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
import math
from tensorflow.keras.models import Model

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

def create_front_extract():
    pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def create_side_extract():
    pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model.trainable = False
    return pretrained_model

def multi_input_generator(front_gen, side_gen):
    while True:
        front_batch, y1 = next(front_gen)
        side_batch, y2 = next(side_gen)
        assert (y1 == y2).all(), "Label mismatch!"  # 確保標籤一致
        yield ([front_batch, side_batch], y1)


# def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):
def train_cross_validation(image_dir, image_dir2, n_splits=5, class_count=2):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================

    ## load data and  labels
    # =========================
    data2 = load_path(image_dir2, class_count)
    labels = []
    filepaths = []
    for row in data2:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images2 = pd.concat([filepaths, labels], axis=1)
    # =========================

    ## split image
    # =========================
    # train_df_front, val_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
    # train_df_front, test_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
    train_df_front = images.sample(frac=1, random_state=42)
    test_df_front = images2
    print("Training set label distribution:\n", train_df_front['Label'].value_counts(normalize=False))
    print("Test set label distribution:\n", test_df_front['Label'].value_counts(normalize=False))
    # =========================

    preprocessing_function_chosen_front = tf.keras.applications.resnet50.preprocess_input
    preprocessing_function_chosen_side = tf.keras.applications.efficientnet.preprocess_input


    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    test_acc = []
    test_f1 = []

    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df_front, train_df_front['Label']):
        k_fold_train_front = train_df_front.iloc[train_index]
        k_fold_val_front = train_df_front.iloc[val_index]

        # front images
        # =========================
        train_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                                preprocessing_function=preprocessing_function_chosen_front)
        test_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_front)

        train_images_front = train_generator_front.flow_from_dataframe(
            dataframe=k_fold_train_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        val_images_front = train_generator_front.flow_from_dataframe(
            dataframe=k_fold_val_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        test_images_front = test_generator_front.flow_from_dataframe(
            dataframe=test_df_front,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=32,
            shuffle=False
        )
        # =========================   

        # side images
        # =========================
        train_df_side = train_df_front.copy()
        test_df_side = test_df_front.copy()
        train_df_side.loc[:, "Filepath"] = train_df_front["Filepath"].str.replace("front", "side")
        test_df_side.loc[:, "Filepath"] = test_df_side["Filepath"].str.replace("front", "side")

        k_fold_train_side = train_df_side.iloc[train_index]
        k_fold_val_side = train_df_side.iloc[val_index]

        train_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                                preprocessing_function=preprocessing_function_chosen_side)
        test_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_side)

        train_images_side = train_generator_side.flow_from_dataframe(
            dataframe=k_fold_train_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        val_images_side = train_generator_side.flow_from_dataframe(
            dataframe=k_fold_val_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=64,
            shuffle=False,
            seed=42
        )

        test_images_side = test_generator_side.flow_from_dataframe(
            dataframe=test_df_side,
            x_col='Filepath',
            y_col='Label',
            target_size=(224, 224),
            color_mode='rgb',
            class_mode='categorical',
            batch_size=32,
            shuffle=False
        )
        # ========================= 
        
        # load model
        # =========================
        # input 
        input_front = Input(shape=(224, 224, 3), name="AP(Mortise)")
        input_side = Input(shape=(224, 224, 3), name="Lateral")

        # model
        model_front = create_front_extract()
        model_side = create_side_extract()

        features_front = model_front(input_front)
        features_side = model_side(input_side)

        features_front_x = tf.keras.layers.Dense(128, activation='relu', name='AP_dense_128')(features_front)
        features_front_x = tf.keras.layers.Dense(50, activation='relu', name='AP_dense_50')(features_front_x)

        features_side_x = tf.keras.layers.Dense(128, activation='relu', name='Lateral_dense_128')(features_side)
        features_side_x = tf.keras.layers.Dense(50, activation='relu', name='Lateral_dense_50')(features_side_x)

        fused_features = Concatenate(name="feature_fusion")([features_front_x, features_side_x])
        # fused_features = tf.keras.layers.Dense(50, activation='relu', name='fusion_dense_50')(fused_features)
        fused_features = Dropout(0.1)(fused_features)

        final_output = Dense(class_count, activation='sigmoid', name='output_layer')(fused_features)
        multi_view_model = None
        multi_view_model = Model(
            inputs=[input_front, input_side],
            outputs=final_output
        )
        multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        multi_view_model.summary()
        # print(model.summary())
        # =========================

        train_generator = multi_input_generator(train_images_front, train_images_side)
        val_generator = multi_input_generator(val_images_front, val_images_side)
        
        ## compile and evaluate
        # =========================

        print("-------Training_" + concat_type + "-------")
        multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
        
        batch_size = 64
        ## early stop 
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
        multi_view_model.fit(train_generator, validation_data=val_generator, callbacks=[early_stopping], epochs=30,
                                    steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
                                    validation_steps= math.ceil(val_images_front.samples / batch_size))
        ## no early stop
        # multi_view_model.fit(train_generator, validation_data=val_generator, epochs=30,
        #                     steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
        #                     validation_steps= math.ceil(val_images_front.samples / batch_size))

        # =========================

        test_generator = multi_input_generator(test_images_front, test_images_side)
        batch_size=32
        pred = multi_view_model.predict(test_generator,  steps=math.ceil(test_images_front.samples / batch_size))
        predicted_labels = np.argmax(pred, axis=1)

        # =========================
            
        ## print results
        # =========================
        acc = accuracy_score(test_images_front.labels, predicted_labels)

        test_acc.append(np.round(acc, 2))
        f1 = f1_score(test_images_front.labels, predicted_labels, average='macro')
        test_f1.append(np.round(f1, 2))
        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    # =========================
    acc_mean  = np.round(np.mean(test_acc), 2)
    acc_std = np.round(np.std(test_acc), 2)
    f1_mean  = np.round(np.mean(test_f1), 2)
    f1_std = np.round(np.std(test_f1), 2)
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':test_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':test_f1}

# =========================




results = []
results = train_cross_validation(image_dir=path,image_dir2=path2, class_count=3)


print(f"acc mean = {results['acc_mean']}, std = {results['acc_std']}, test_acc = {results['test_acc']}")
print(f"f1  mean = {results['f1_mean']}, std = {results['f1_std']}, test_f1 = {results['test_f1']}")
  

Training set label distribution:
 0    126
2     87
1     84
Name: Label, dtype: int64
Test set label distribution:
 2    35
0    34
1    31
Name: Label, dtype: int64
Found 237 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Found 237 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.
Found 100 validated image filenames belonging to 3 classes.
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AP(Mortise) (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Lateral (InputLayer)            [(None, 224, 224, 3) 0       